### Imports

In [1]:
import glob
import os
import shutil
import tensorflow 
from tensorflow.keras import Sequential,models
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Conv2D,MaxPooling2D, Dropout, GlobalAveragePooling2D,Activation
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-04 09:12:37.481632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


### Constants

In [4]:
# Constants which are used below
BATCH_SIZE = 5
IMG_SIZE_DIM = 150

### Generators

Without augmentation... it just take data from given path and then create batch.

In [22]:
# train_dir = '../input/trafficsignmodified/trafficDataOriginal/trafficDataOriginal/DATA'
# test_dir = '../input/trafficsignmodified/trafficDataOriginal/trafficDataOriginal/TEST'
# # No specific val class... we take it randomly from train folder
# aug = ImageDataGenerator(
#   rescale=1./255,
# 	rotation_range=10,
# 	zoom_range=0.15,
# 	width_shift_range=0.1,
# 	height_shift_range=0.1,
# 	shear_range=0.15,
# 	horizontal_flip=False,
# 	vertical_flip=False,
# 	fill_mode="nearest",
#     validation_split=0.20)

# test_datagen = ImageDataGenerator(rescale=1./255)

# train_ds = aug.flow_from_directory(
#         directory = train_dir,
#         target_size=(IMG_SIZE_DIM, IMG_SIZE_DIM),
#         batch_size=BATCH_SIZE,
#         subset='training',
#         class_mode="categorical",
#         shuffle=True,
#         seed=42)

# val_ds = aug.flow_from_directory(
#         directory = train_dir,
#         target_size=(IMG_SIZE_DIM, IMG_SIZE_DIM),
#         batch_size=BATCH_SIZE,
#         subset='validation',
#         class_mode="categorical",
#         shuffle=True,
#         seed=42)

# test_ds = test_datagen.flow_from_directory(
#         directory = test_dir,
#         target_size = (IMG_SIZE_DIM,IMG_SIZE_DIM),
#         batch_size = BATCH_SIZE,
#         shuffle=False,
#         seed=42)

Found 3359 images belonging to 58 classes.
Found 811 images belonging to 58 classes.
Found 1994 images belonging to 54 classes.


### Generator With Augmentation

In [5]:
test_dir = '../input/trafficsignmodified/trafficDataRestructuredAsJDNamedFolder/trafficDataRestructuredAsJDNamedFolder/test'
train_dir = '../input/trafficsignmodified/trafficDataRestructuredAsJDNamedFolder/trafficDataRestructuredAsJDNamedFolder/train'
val_dir = '../input/trafficsignmodified/trafficDataRestructuredAsJDNamedFolder/trafficDataRestructuredAsJDNamedFolder/val'

In [6]:
# Augmentation + premade val class

aug = ImageDataGenerator(
    rescale=1./255,
	rotation_range=10,
	zoom_range=0.15,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = aug.flow_from_directory(
        directory = train_dir,
        target_size=(IMG_SIZE_DIM, IMG_SIZE_DIM),
        batch_size=BATCH_SIZE,
        class_mode="sparse",
        shuffle=True,
        seed=42)

val_ds = aug.flow_from_directory(
        directory = val_dir,
        target_size=(IMG_SIZE_DIM, IMG_SIZE_DIM),
        batch_size=BATCH_SIZE,
        class_mode="sparse",
        shuffle=True,
        seed=42)

test_ds = test_datagen.flow_from_directory(
        directory = test_dir,
        target_size = (IMG_SIZE_DIM,IMG_SIZE_DIM),
        batch_size = BATCH_SIZE,
        class_mode="sparse",
        shuffle=False,
        seed=42)


Found 4794 images belonging to 58 classes.
Found 857 images belonging to 58 classes.
Found 517 images belonging to 58 classes.


### Start Creating Model VGG16

In [7]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(IMG_SIZE_DIM,IMG_SIZE_DIM,3)
)

2022-11-04 09:13:02.611849: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-04 09:13:02.615658: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-04 09:13:02.681249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 09:13:02.681925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-11-04 09:13:02.681977: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-04 09:13:02.714950: I tensorflow/stream_executor/platform/def

58892288/58889256 [==============================] - 0s 0us/step


In [8]:
from tensorflow.keras.layers import BatchNormalization

In [9]:

model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(58,activation='softmax'))

In [12]:
model.compile(optimizer='adam',loss = "sparse_categorical_crossentropy",metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="/kaggle/working/VGG16_150-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
model = models.load_model('../input/trafficsignmodified/VGG16_150-40-0.96_sparse.hdf5')

In [15]:
history = model.fit(train_ds,epochs=40,validation_data=val_ds,callbacks=callbacks_list)

2022-11-04 09:14:11.989110: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-04 09:14:11.993470: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000150000 Hz


Epoch 1/40


2022-11-04 09:14:12.721930: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-11-04 09:14:13.550810: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-11-04 09:14:13.585579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


959/959 [==============================] - 69s 64ms/step - loss: 0.4285 - accuracy: 0.8776 - val_loss: 47.4972 - val_accuracy: 0.9650

Epoch 00001: val_accuracy improved from -inf to 0.96499, saving model to /kaggle/working/VGG16_150-01-0.96.hdf5
Epoch 2/40
959/959 [==============================] - 50s 52ms/step - loss: 0.4000 - accuracy: 0.8876 - val_loss: 38.1080 - val_accuracy: 0.9627

Epoch 00002: val_accuracy did not improve from 0.96499
Epoch 3/40
959/959 [==============================] - 49s 51ms/step - loss: 0.3987 - accuracy: 0.8894 - val_loss: 2406295296.0000 - val_accuracy: 0.9603

Epoch 00003: val_accuracy did not improve from 0.96499
Epoch 4/40
959/959 [==============================] - 51s 53ms/step - loss: 0.3926 - accuracy: 0.8869 - val_loss: 5261732.5000 - val_accuracy: 0.9662

Epoch 00004: val_accuracy improved from 0.96499 to 0.96616, saving model to /kaggle/working/VGG16_150-04-0.97.hdf5
Epoch 5/40
959/959 [==============================] - 50s 53ms/step - loss: 0

In [16]:
model = models.load_model('/kaggle/working/VGG16_150-24-0.99.hdf5') # Loading the best model which is autosaved.

In [18]:
print("Evaluate on test data")
results = model.evaluate(test_ds,batch_size=5)
print("test loss, test acc:", results)

Evaluate on test data
104/104 [==============================] - 3s 24ms/step - loss: 0.0332 - accuracy: 0.9923
test loss, test acc: [0.03324761241674423, 0.9922630786895752]
